## Word2Vec (Global)

In [21]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install scipy
#!{sys.executable} -m pip install gensim==3.8.1

In [22]:
# Load model
from gensim.models import Word2Vec
filename = "./data/model/frwiki/frwiki.gensim"
model = Word2Vec.load(filename)

In [23]:
result = model.most_similar(positive=['sushi', 'france'], negative=['japon'], topn=5)
print(result)

result = model.most_similar(positive=['roi', 'femme'], negative=['homme'], topn=2)
print(result)

C:\Users\massa\AppData\Local\Temp\ipykernel_15948\4228314577.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  result = model.most_similar(positive=['sushi', 'france'], negative=['japon'], topn=5)


[('kebab', 0.4288313090801239), ('cassoulet', 0.4284020662307739), ('saucisson', 0.4272685647010803), ('daube', 0.42331743240356445), ('traiteur', 0.4228731393814087)]
[('reine', 0.6332529783248901), ('régente', 0.5426114797592163)]


C:\Users\massa\AppData\Local\Temp\ipykernel_15948\4228314577.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  result = model.most_similar(positive=['roi', 'femme'], negative=['homme'], topn=2)


In [24]:
# Read json source
import json
from os import path
import pprint

WordData = json.load(open(path.abspath('./data/generated/output/infos.json'), encoding="utf-8"))

all_vectors = []
result = {}
for candidat, infos in WordData.items():
    tfIdfs = dict(list(infos["tf_idfs"].items())[0:50])
    sumTfidf = sum(tfIdfs.values())
    wordVectors = []
    for word, value in tfIdfs.items():
        wordVectors.append((value/sumTfidf) * model[word])

    candidat_vector = wordVectors[0]
    for vector in wordVectors[1:]:
        candidat_vector = candidat_vector + vector
    candidat_vector = candidat_vector / len(wordVectors)

    all_vectors.append(candidat_vector)

    result[candidat] = {}
    result[candidat]['vector'] = candidat_vector
    result[candidat]['notions_globales'] = model.most_similar(positive=wordVectors, topn=10)



C:\Users\massa\AppData\Local\Temp\ipykernel_15948\380424935.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  wordVectors.append((value/sumTfidf) * model[word])
C:\Users\massa\AppData\Local\Temp\ipykernel_15948\380424935.py:26: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  result[candidat]['notions_globales'] = model.most_similar(positive=wordVectors, topn=10)


In [25]:
# Calcul du vecteur moyen
vector_global = all_vectors[0]
for vector in all_vectors[1:]:
    vector_global = vector_global + vector

vector_global  = vector_global / len(all_vectors)

# Calcul du vecteur différenciant de chaque candidat
for candidat, infos in result.items():
    vector_candidat = infos['vector']
    result[candidat]['notions_specifiques'] = model.most_similar(positive=[vector_candidat], negative=[vector_global], topn=10)

C:\Users\massa\AppData\Local\Temp\ipykernel_15948\2342175415.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  result[candidat]['notions_specifiques'] = model.most_similar(positive=[vector_candidat], negative=[vector_global], topn=10)


In [26]:
pprint.pprint(result)

import json
import os
OUTPUT_DIR_PATH = path.abspath('./data/generated/output/')
output_filepath = path.join(OUTPUT_DIR_PATH, 'infos_vectorization.json')

if not os.path.exists(OUTPUT_DIR_PATH):
    os.makedirs(OUTPUT_DIR_PATH)
with open(output_filepath, 'w', encoding='utf-8') as file:
    json.dump(result, file, ensure_ascii=False, indent=2)

{'dupont-aignan': {'notions_globales': [('économique', 0.5779117345809937),
                                        ('développement', 0.5669440031051636),
                                        ('économie', 0.5376861095428467),
                                        ('santé', 0.5209861397743225),
                                        ('entreprises', 0.5162258148193359),
                                        ('investissement', 0.5116778612136841),
                                        ('salariés', 0.5103652477264404),
                                        ('gestion', 0.5054322481155396),
                                        ('environnement', 0.503185510635376),
                                        ('sociale', 0.4993606209754944)],
                   'notions_specifiques': [('développer', 0.4345000088214874),
                                           ('accroitre', 0.3620188236236572),
                                           ('développement',
                          